### Performance Evaluation: AP scores ###

In [1]:
import sys
import os
import numpy as np
import json
import pandas as pd
import logging
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

logger = logging.getLogger(__name__)

# PyTorch
import torch
from torch.utils.data import DataLoader
from torchvision import ops

# Hugging Face Library
from transformers import RTDetrV2ForObjectDetection, RTDetrImageProcessor

%load_ext autoreload
%autoreload 2
import computervision
from computervision.imageproc import is_image, ImageData, clipxywh, xyxy2xywh, xywh2xyxy
from computervision.datasets import DETRdataset, get_gpu_info
from computervision.transformations import AugmentationTransform
from computervision.performance import DetectionMetrics
from computervision.inference import DETR

print(f'Project version: {computervision.__version__}')
print(f'Python version:  {sys.version}')

Project version: v0.0.1
Python version:  3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [2]:
# Check GPU availability
device, device_str = get_gpu_info()
print(f'Current device {device}')

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3060 Laptop GPU
PyTorch version:   2.8.0a0+34c6371d24.nv25.08
CUDA version:      13.0
CUDNN version:     91200
Device for model training/inference: cuda:0
Current device cuda:0


In [3]:
# Function to plot an image with the bounding boxes
def plot_boxes(image, box_list, ax, label_list=None, color=None, cmap='grey', offset_xy=(0, 0)):
    offset_xy = (10 + offset_xy[0], 100 + offset_xy[1])
    # Take a ratio that looks good
    offset = (image.shape[1]*offset_xy[0]/2500,
              image.shape[0]*offset_xy[1]/1250)
    if color is None:
        # If no color is provided, color each box in a different color
        color_list = list(plt.cm.rainbow(np.linspace(0, 1, len(box_list))))
    else:
        color_list = [color]*len(box_list)
    ax.set(xticks=[], yticks=[])
    ax.imshow(image, cmap=cmap)
    # Loop over the bounding boxes
    for b, box in enumerate(box_list):
        rect = Rectangle(xy=(box[0], box[1]),
                         width=box[2],
                         height=box[3],
                         linewidth=1.5,
                         edgecolor=color_list[b],
                         facecolor='none',
                         alpha=0.7)
        ax.add_patch(rect)
        if label_list is not None:
            ax.text(x=box[0]+offset[0], y=box[1]+offset[1], s=label_list[b], color=color_list[b], fontsize=8)
    return ax

### Test data ###

In [ ]:
# HSDM DATA
image_dir = os.path.join(os.environ.get('DATA'), 'dataset_object_240921')
df_file_name = 'objectdata_240921_clean_ide.parquet'
df_file = os.path.join(image_dir, df_file_name)
df = pd.read_parquet(df_file)
display(df.head(2))

file_col = 'multi_file'
pos_col = 'pos'
bbox_col = 'bbox'

file_name_list = sorted(list(df[file_col].unique()))
file_list = [os.path.join(image_dir, file) for file in file_name_list]
# Check the test data
checked = [is_image(file) for file in file_list]
assert np.sum(checked) == len(file_list)

In [4]:
# Dentex testing data
image_dir = os.path.join(os.environ.get('DATA'), 'dentex_detection_250928', 'test')
df_file_name = 'train_quadrant_enumeration_test_set.parquet'
df_file = os.path.join(image_dir, df_file_name)
df = pd.read_parquet(df_file)
df = df.loc[
    (df['dset'] == 'test') & 
    (df['quadrants'].isin([14, 23])) &
    (df['transformation'].isin(range(2)))]

print(f'Number of images in Dentex test set: {len(df['file_name'].unique())}')
print(f'Annotations in Dentex test set:      {df.shape[0]}')

file_col = 'file_name'
pos_col = 'ada'
bbox_col = 'bbox'

df = df.astype({pos_col: int})
display(df.head(2))

file_name_list = sorted(list(df[file_col].unique()))
file_list = [os.path.join(image_dir, file) for file in file_name_list]
# Check the test data
checked = [is_image(file) for file in file_list]
assert np.sum(checked) == len(file_list)

Number of images in Dentex test set: 80
Annotations in Dentex test set:      895


,bbox,quadrant,ada,file_name,file_base_name,quadrants,height,width,transformation,transformation_name,dset
11615,"[424, 0, 94, 314]",1.0,6,test_train_219_14_00.png,train_219,14,519,639,0,test_set,test
11616,"[311, 0, 160, 283]",1.0,5,test_train_219_14_00.png,train_219,14,519,639,0,test_set,test


In [ ]:
# Roboflow test data
image_dir = os.path.join(os.environ.get('DATA'), 'dataset_object_roboflow_240930')
df_file_name = 'roboflow_240930.parquet'
df_file = os.path.join(image_dir, df_file_name)
df = pd.read_parquet(df_file)
display(df.head(2))
# What are the categories in this data set
print(list(df['category'].unique()))

file_col = 'multi_file'
pos_col = 'pos'
bbox_col = 'bbox'

# Let's get rid of the rows that do not have locations
df = df.loc[~df[pos_col].isnull()]

file_name_list = sorted(list(df[file_col].unique()))
file_list = [os.path.join(image_dir, file) for file in file_name_list]
# Check the test data
checked = [is_image(file) for file in file_list]
assert np.sum(checked) == len(file_list)
print(f'Using {len(checked)} images from the Roboflow data set')
print(f'Annotations: {df.shape[0]}')

### Model and image processor ###

In [5]:
model_name = 'rtdetr_251001_02'
model_dir = os.path.join(os.environ.get('DATA'), 'model', model_name)
model_json_name = f'{model_name}.json'
model_json_file = os.path.join(model_dir, model_json_name)

checkpoint_name = 'checkpoint-17500'
checkpoint = os.path.join(model_dir, checkpoint_name)

# Load model parameters
with open(model_json_file, mode='r') as fl:
    model_parameters = json.load(fl)
display(model_parameters.keys())

# Load the model
dtr = DETR(checkpoint_path=checkpoint)

model_info = model_parameters.get('model_info')
print()
display(model_info)

dict_keys(['model_info', 'id2label', 'training_args', 'processor_params', 'bbox_format'])

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3060 Laptop GPU
PyTorch version:   2.8.0a0+34c6371d24.nv25.08
CUDA version:      13.0
CUDNN version:     91200
Device for model training/inference: cuda:0



{'model_version': 2,
 'device_number': 2,
 'project_version': 'v0.0.1',
 'model_name': 'rtdetr_251001_02',
 'train_image_dir': '/app/data_model/dentex_detection_250928',
 'val_image_dir': '/app/data_model/dentex_detection_250928/test',
 'model_dir': '/app/data_model/model',
 'im_width': 640,
 'im_height': 640,
 'hf_checkpoint': 'PekingU/rtdetr_v2_r101vd',
 'training_checkpoint': 'PekingU/rtdetr_v2_r101vd',
 'train_quadrants': [14, 23],
 'val_quadrants': [14, 23],
 'train_transform_name': 'train_14_23',
 'val_transform_name': 'val'}

In [6]:
# Category ID to label conversion
id2label = model_parameters.get('id2label')
id2label = {int(k): int(v) for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

### Run the test data through the model ###

In [7]:
# Validation transform
transform = AugmentationTransform(im_width=model_info.get('im_width'), 
                                  im_height=model_info.get('im_height')).\
    get_transforms(name=model_info.get('val_transform_name'))

df = df.assign(label=df[pos_col].apply(lambda ada: label2id.get(ada)))

test_dataset = DETRdataset(data=df.copy(), 
                           image_processor=dtr.processor,
                           image_dir=image_dir,
                           file_name_col=file_col,
                           label_id_col='label',
                           bbox_col=bbox_col,
                           bbox_format=model_parameters.get('bbox_format'),
                           transforms=transform)

In [9]:
def collate_fn(batch):
    """
    Collates a batch of data samples into a single dictionary for model input.
    """
    data = {}
    data["pixel_values"] = torch.stack([x["pixel_values"] for x in batch])
    data["labels"] = [x["labels"] for x in batch]
    return data

In [15]:
model = dtr.model
model.eval()
data_loader = DataLoader(dataset=test_dataset, 
                         batch_size=4, 
                         collate_fn=collate_fn,
                         pin_memory=True)

with torch.no_grad():
    for b, batch in enumerate(data_loader):
        print(f'Loading batch {b + 1}')
        batch['pixel_values'].to(device)
        batch['labels'].to(device)
        #outputs = model(**batch)

Loading batch 1


AttributeError: 'list' object has no attribute 'to'

In [14]:
batch.keys()

dict_keys(['pixel_values', 'labels'])

In [ ]:
test_dataset[10]

In [ ]:
figsize = (4, 4)
threshold = 0.02
idx = 11
file_name = file_name_list[idx]
    
file = os.path.join(image_dir, file_name)
im = ImageData().load_image(file)
im = ImageData().np2color(im)
df_file = df.loc[df[file_col] == file_name]
bbox_list = df_file[bbox_col].tolist()
pos_list = df_file[pos_col].tolist()
pos_list = sorted([int(pos) for pos in pos_list])
    
# Predict positions
output = dtr.predict(image=im, threshold=threshold)
if output is not None:
    output_bbox_list = output.get('bboxes')
    output_cat_list = output.get('categories')
    output_pos_list = [id2label.get(cat) for cat in output_cat_list]
    score_list = output.get('scores')
    output_label_list = [f'P{p}\n{s:.2f}' for p, s in zip(output_pos_list, score_list)]

    print(f'True: {pos_list}')
    print(f'Pred: {sorted(output_pos_list)}')
    
    fig, ax = plt.subplots(figsize=figsize)
    ax = plot_boxes(image=im, box_list=bbox_list, label_list=pos_list, color='w', ax=ax)
    ax = plot_boxes(image=im, box_list=output_bbox_list, label_list=output_label_list, ax=ax, offset_xy=(0, 200))
    plt.show()

In [ ]:
# Let's pick one ground truth and one predicted position
# So we can calculate the iou
true_pos = 12
true_bbx = list(bbox_list[pos_list.index(true_pos)])
true_bbx = clipxywh(true_bbx, xlim=(0, im.shape[1]), ylim=(0, im.shape[0]), decimals=0)
print(true_bbx)

pred_pos_idx = output_pos_list.index(true_pos)
pred_pos = output_pos_list[pred_pos_idx]
print(pred_pos)
pred_bbx = output_bbox_list[pred_pos_idx]
print(pred_bbx)

# Plot this situation
fig, ax = plt.subplots(figsize=figsize)
ax = plot_boxes(image=im, box_list=[true_bbx], label_list=[true_pos], ax=ax, color='w')
ax = plot_boxes(image=im, box_list=[pred_bbx], label_list=[pred_pos], ax=ax, color='r', offset_xy=(0, 20))
plt.show()

# Calculate the IoU for the predicted bounding box and the ground truth
iou = DetectionMetrics.compute_iou(bbox_1=true_bbx, bbox_2=pred_bbx, bbox_format='xywh', method='pt')
print(f'IoU for label {true_pos}: {iou}')

In [ ]:
# Performance evaluations
score_threshold = 0.02
iou_threshold = 0.5

# Ground truth
true_labels = [int(l) for l in df_file[pos_col].tolist()]
true_bboxes = df_file[bbox_col].tolist()
true_bboxes = [clipxywh(list(box), xlim=x_lim, ylim=y_lim, decimals=0) for box in true_bboxes]
print(f'True labels:        {true_labels}')

# Predictions
output = dtr.predict(image=im, threshold=score_threshold)
pred_labels = [int(id2label.get(cat)) for cat in output.get('categories')]
pred_bboxes = output.get('bboxes')
pred_bboxes = [clipxywh(list(box), xlim=x_lim, ylim=y_lim, decimals=0) for box in pred_bboxes]
pred_scores = output.get('scores')
print(f'Pred labels:        {pred_labels}')

# Determine the false negative samples
# Predictions that were missed by the object detection
missed = sorted(list(set(true_labels).difference(pred_labels)))
print(f'Missed predictions: {missed}')

# We want to classify the predictions
iou_list = []
prediction_list = []
for p, p_label in enumerate(pred_labels):
    
    p_bbox = pred_bboxes[p]
    
    # When there is no label for this prediction, it is a FP prediction
    # NOTE: the FP numbers will be too high if not all of the instances are labeled!
    p_prediction = 'FP'
    p_iou = np.nan
    
    # Let's check each ground truth labels if we have a match
    pt_iou_list = []
    t_bbx_list = []
    
    for t, t_label in enumerate(true_labels):
        if p_label == t_label:
            t_bbox = true_bboxes[t]
            pt_iou = DetectionMetrics.compute_iou(p_bbox, t_bbox, bbox_format='xywh', method='pt')
            pt_iou_list.append(pt_iou)
            t_bbx_list.append(t_bbox)
            
    if len(pt_iou_list) > 0:
        p_iou = np.max(pt_iou_list)
        t_bbx = t_bbx_list[np.argmax(pt_iou_list)]
        if p_iou >= iou_threshold:
            p_prediction = 'TP'
        
        print(f'Prediction for label {p + 1}/{len(pred_labels)}: {p_label}: {p_prediction} with IoU: {p_iou}')
        
        # Plot the situation when we have a match
        fig, ax = plt.subplots(figsize=figsize)
        ax = plot_boxes(image=im, box_list=[t_bbx], label_list=[p_label], ax=ax, color='w')
        ax = plot_boxes(image=im, box_list=[p_bbox], label_list=[p_label], ax=ax, color='r', offset_xy=(0, 20))
        plt.show()

    iou_list.append(p_iou)
    prediction_list.append(p_prediction)

# What if we predict the same class twice and both are TP?
# We count only the first one as a TP, the others we set to FP.
c = pd.DataFrame({'pred_label': pred_labels,
                  'TP': prediction_list,
                  'score': pred_scores,
                  'IoU': iou_list})

c = c.assign(duplicate_TP=False)

# Flip duplicate TP predictions
d = c.copy()
d.loc[(c.duplicated(subset=['pred_label', 'TP'])) & (c['TP'] == 'TP'), 'TP'] = 'FP'
d.loc[(c.duplicated(subset=['pred_label', 'TP'])) & (c['TP'] == 'TP'), 'duplicate_TP'] = True

print(pred_labels)
print(true_labels)
print(iou_list)
print(prediction_list)
display(d)

### Prediction classification method ###

In [ ]:
print(f'True labels: {true_labels}')
print(f'Pred labels: {pred_labels}')

missed, predictions_df = DetectionMetrics.\
    classify_predictions(true_labels=true_labels, 
                         true_bboxes=true_bboxes, 
                         pred_labels=pred_labels, 
                         pred_bboxes=pred_bboxes, 
                         iou_threshold=iou_threshold)

print(missed)
display(predictions_df)

In [ ]:
# Run inference on the entire data set
figsize = (4, 4)
score_threshold = 0.02
iou_threshold = 0.5

idx = 123
file_name = file_name_list[idx]
file = os.path.join(image_dir, file_name)
im = ImageData().load_image(file)
im = ImageData().np2color(im)
x_lim = (0, im.shape[1])
y_lim = (0, im.shape[0])
df_file = df.loc[df[file_col] == file_name]
bbox_list = [clipxywh(list(box), xlim=x_lim, ylim=y_lim, decimals=0) for box in df_file[bbox_col].tolist()]
pos_list = [int(pos) for pos in df_file[pos_col].tolist()]
    
# Prediction
output = dtr.predict(image=im, threshold=score_threshold)
if output is not None:
    output_bbox_list = output.get('bboxes')
    output_cat_list = output.get('categories')
    output_pos_list = [id2label.get(cat) for cat in output_cat_list]
    score_list = output.get('scores')
    output_label_list = [f'P{p}\n{s:.2f}' for p, s in zip(output_pos_list, score_list)]

    print(f'True: {pos_list}')
    print(f'Pred: {output_pos_list}')

    # Classify predictions
    missed, pred_df = DetectionMetrics.classify_predictions(true_labels=pos_list,
                                                            true_bboxes=bbox_list,
                                                            pred_labels=output_pos_list,
                                                            pred_bboxes=output_bbox_list)
    print(f'Missed: {missed}')
    display(pred_df)

    # Calculate precision/recall
    
    
    fig, ax = plt.subplots(figsize=figsize)
    ax = plot_boxes(image=im, box_list=bbox_list, label_list=pos_list, color='w', ax=ax)
    ax = plot_boxes(image=im, box_list=output_bbox_list, label_list=output_label_list, ax=ax, offset_xy=(0, 200))
    plt.show()